In [ ]:
# Libraries

import numpy as np
from skimage.transform import resize
import pymartini
import rasterio
import pyvista as pv
from scipy.ndimage import map_coordinates
import settings
import utils

# matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib as matplot
# plotting
import seaborn as sns
# for interactions
from mpl_toolkits.mplot3d import Axes3D

import TIN_engine
from TIN_engine import *
from TIN_draw import *
from TIN_drainage import *
from TIN_watershed import *
from TIN import *

# remove this (and possibly restart the kernel) if you don't want interactive plots
%matplotlib widget

# for reloading modules (specifically TIN_engine) during development
%load_ext autoreload
%autoreload 2


# pysheds
from pysheds.grid import Grid

# other
import time
import copy

In [ ]:
DATA_FILE = settings.WASHINGTON_SMALL
WATERSHED_POUR_POINT = (-116.574585, 49.49625)

In [ ]:
# ---- Open Rasters ----
grid = Grid.from_raster(str(DATA_FILE))
dem = grid.read_raster(str(DATA_FILE))

# Watershed Delineation With TIN

In [ ]:
# ---- Load DEM and create TIN ----
vertices_3d, triangles, xs, ys, zs, zs_scaled  = get_triangles_from_DEM(DATA_FILE, mesh_level=90)

In [ ]:
# ---- Get Highest Point ----
max_index = np.nanargmax(zs)
max_index = np.nanargmin(zs)
highest_point = [xs[max_index], ys[max_index], zs[max_index]]
highest_point

In [ ]:
# ---- Get Subset of Triangles Around Highest Point ----
radius = 1
bounds_around_highest = [highest_point[0] + radius, highest_point[1] + radius, highest_point[0] - radius, highest_point[1] - radius]

triangles_subset = get_subset_of_triangles_from_bounds(triangles, bounds_around_highest, xs, ys)

In [ ]:
# ---- Convert to Triangle and Vertex Objects ----
triangle_objects, vertices = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)
print(len(triangle_objects), "triangle objects created.")
print(len(vertices), "vertex objects created.")

In [ ]:
# ---- Preprocessing ----
has_flat_triangles(triangle_objects)
flat = get_flat_triangles(triangle_objects)
len(flat)
unflaten_triangles(triangle_objects)

In [ ]:
# ~16 seconds for 18000 triangles

# ---- Drainage network calculation ----
drainage_outlet_nodes = create_drainage_network(triangle_objects)
print(len(drainage_outlet_nodes), "outlet nodes created.")

In [ ]:
# ---- Watershed delineation ----
watershed, start_node = delineate_watershed(drainage_outlet_nodes, highest_point)

print(len(watershed), "triangles")

In [ ]:
start_node.point
WATERSHED_POUR_POINT = (start_node.point[0], start_node.point[1])
WATERSHED_POUR_POINT

In [ ]:
# ---- Visualize watershed ----

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors

def draw_node(node: Node, depth, color):
    for upstream_node in node.upstream_nodes:
        draw_line_points(ax, node.point[0:2], upstream_node.point[0:2], color, linewidth=2)
        draw_node(upstream_node, depth + 1, color)

SHOW_DRAINAGE_NETWORK = True
SHOW_TIN_TRIANGLES = False

fig = plt.figure()
ax = fig.add_subplot(111)

# draw dem heat map
ax.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)

# draw triangles
if SHOW_TIN_TRIANGLES:
    norm = mcolors.Normalize(vmin=zs.min(), vmax=zs.max()) 
    scale_map = cm.ScalarMappable(cmap=cm.terrain, norm=norm)

    for triangle in triangle_objects:
        color = scale_map.to_rgba(triangle.get_centroid().z)
        draw_triangle_object(ax, triangle, color)

    print("Finished drawing terrain triangles.")

# draw watershed
for triangle in watershed:
    draw_triangle_object(ax, triangle, "#00ff0055", filled=True)

print("Done drawing watershed triangles.")

# draw drainage network
if SHOW_DRAINAGE_NETWORK:
    draw_point(ax, start_node.point[0:2], markersize=4)
    random_color = np.random.rand(3,)
    draw_node(start_node, 0, random_color)

    print("Done drawing drainage network.")

plt.show()

# Watershed Delineation With Raster

In [ ]:
# ---- Plot the DEM ----
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)
plt.colorbar(label='Elevation (m)')
plt.grid(zorder=0)
plt.title('Digital elevation map', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

In [ ]:
# ---- Determine D8 flow directions from DEM ----

# ESRI scheme that specifies directions as numbers (which is the default)
"""
North: 64
Northeast: 128
East: 1
Southeast: 2
South: 4
Southwest: 8
West: 16
Northwest: 32
"""

#dirmap = (64, 128, 1, 2, 4, 8, 16, 32) # this is the default value
dirmap = (7, 8, 1, 2, 3, 4, 5, 6) # new value to make the D8 and Dinf plots the same
    
# Compute flow directions using D8
fdir = grid.flowdir(dem, dirmap=dirmap)

In [ ]:
# ---- Flow Accumulation ----
acc = grid.accumulation(fdir, dirmap=dirmap)

In [ ]:
# ---- Delineate a Catchment ----

# Specify pour point
#x, y = -97.294, 32.737
#x, y = -113.0495, 47.2395
#x, y = -114.0, 48.0

# mid point
x = ((grid.bbox[2] - grid.bbox[0]) / 2) + grid.bbox[0]
y = ((grid.bbox[3] - grid.bbox[1]) / 2) + grid.bbox[1]

# Snap pour point to high accumulation cell
#x_snap, y_snap = grid.snap_to_mask(acc > 10, WATERSHED_POUR_POINT)
x_snap, y_snap = WATERSHED_POUR_POINT


# Delineate the catchment
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, 
                       xytype='coordinate')


# Clip the bounding box to the catchment
#clipped_grid = copy.deepcopy(grid) # deep copy so that we can reuse grid if desired
#clipped_grid.clip_to(catch)
#clipped_catch = clipped_grid.view(catch)
#grid_use = clipped_grid

# for no clipping
clipped_catch = catch
grid_use = grid


In [ ]:
# Plot the catchment
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

# draw dem heat map
ax.imshow(dem, extent=grid_use.extent, cmap='terrain', zorder=1)

#plt.grid('on', zorder=0)
im = ax.imshow(np.where(clipped_catch, clipped_catch, np.nan), extent=grid_use.extent,
               zorder=1, cmap='Greys_r')

# this shows the pour point
plt.scatter([x_snap], [y_snap], c='red', s=50, marker='o')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Delineated Catchment', size=14)